# ZAMG Data Hub: 10 Minuten Daten

Quellen:
- **Messstationen Zehnminutendaten**: https://data.hub.zamg.ac.at/dataset/klima-v1-10min
- **API Client:** https://dataset.api.hub.zamg.ac.at/app/frontend/station/historical/klima-v1-10min?anonymous=true

In [ ]:
import pandas as pd, datetime as dt, numpy as np, requests as req
import matplotlib.pyplot as plt
import bz2
pd.set_option('display.max_rows', 128)

In [ ]:
def generate_url(date_from, date_to, station_ids, parameters):
    get_params = {"parameters": ",".join(parameters), "start": date_from,
                "end":date_to, "station_ids": ",".join([str(val) for val in station_ids]),
                "output_format":"csv"}
    return "https://dataset.api.hub.zamg.ac.at/v1/station/historical/klima-v1-10min?"+'&'.join([f"{key}={val}" for key, val in get_params.items()])

In [ ]:
metadata = req.get("https://dataset.api.hub.zamg.ac.at/v1/station/historical/klima-v1-10min/metadata").json()
params = pd.DataFrame(metadata.get("parameters")).astype(
    {"name": "string", "long_name": "string", "desc": "string", "unit": "string"})
stations = pd.DataFrame(metadata.get("stations")).astype(
    {"type": "string", "id": int, "group_id": "Int64", "name": "string", "state": "string",
     "lat": float, "lon": float, "altitude": float, "valid_from": "datetime64",
     "valid_to": "datetime64", "has_sunshine": bool, "has_global_radiation": bool, "is_active": bool})

In [ ]:
station_ids = stations.loc[stations.name == "GUMPOLDSKIRCHEN", "id"].sort_values()
parameters = ["TL", "TLMAX", "TLMIN", "TP", "RF", "DD", "FF", "FFX", "P", "P0", "RR", "RRM", "SO"]
download_url = generate_url("2022-01-01T00:00:00Z", "2022-12-31T23:50:00Z", station_ids, parameters)
dtypes = {"station": int}
dtypes.update({val:float  for val in parameters})
measurements = pd.read_csv(download_url, sep=",", dtype=dtypes, parse_dates=["time"]).query("TL.notna()")

In [ ]:
measurements.to_parquet("gumpoldskirchen_2022.parquet", compression="brotli")

In [ ]:
measurements.set_index("time")[["TLMAX"]].plot.line();